In [7]:
import numpy as np
import pandas as pd
import sqlite3 as sq
import time
import math
import re
from sklearn import preprocessing
import datetime

In [ ]:
#just reading data
con = sq.connect("database.sqlite")
team_atts = pd.read_sql_query("SELECT * from Team_Attributes", con)
teams = pd.read_sql_query("SELECT * from Team", con)
matches = pd.read_sql_query("SELECT * from Match", con)

original_matches = matches.copy()

matches = matches[['date', 'season', 'home_team_goal', 'away_team_goal', 'home_team_api_id', 'away_team_api_id', 
                  'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession',
                  'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD',
                  'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD',
                  'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA']]

attr_dates = pd.DataFrame(columns=['date', 'year', 'month'])
attr_dates['date'] = pd.to_datetime(team_atts['date'])
attr_dates['year'] = [attr_date.year for attr_date in attr_dates['date']]
attr_dates['month'] = [attr_date.month for attr_date in attr_dates['date']]

In [ ]:
buildUpPlaySpeed = {'Slow': 0, 'Balanced': 1, 'Fast': 2}

st = time.time()

denom = len(matches.columns.values)


### Add blank columns for team attributes to be filled in for each match
for column in list(team_atts.columns.values):
    matches['__home_' + column] = np.nan
    
for column in list(team_atts.columns.values):
    matches['__away_' + column] = np.nan

### To assist in filling values later (note the underscores leading __underscoes added above & used here 
### so we don't collide with existing column names)
home_column_indexes = [matches.columns.get_loc('__home_' + col_name) for col_name in team_atts.columns.values]
away_column_indexes = [matches.columns.get_loc('__away_' + col_name) for col_name in team_atts.columns.values]
indexes_to_drop = []

## Part of experiments described below
n_15_none_match = 0
n_15_none_team_att = 0

for index, match in matches.iterrows():
    ### For each match, we find the home and away team for the correct year, and add their data to the 
    ### dataframe
    match_years = [int(year) for year in match['season'].split('/')]
    home_team_id = match['home_team_api_id']
    away_team_id = match['away_team_api_id']
    home_team_atts = team_atts.loc[team_atts['team_api_id'] == home_team_id]
    away_team_atts = team_atts.loc[team_atts['team_api_id'] == away_team_id]
    
    def date_matches(match_years):
        return (((attr_dates['year'] == match_years[0]) & (attr_dates['month'] >= 7)) | 
                ((attr_dates['year'] == match_years[1]) & (attr_dates['month'] < 7)))

    home_team_att = home_team_atts.loc[date_matches(match_years)]
    away_team_att = away_team_atts.loc[date_matches(match_years)]
    
    
    ### This is just an experiment to determine a threshold for how many values should be 'None'
    ### in match data in order for us to drop a row. To drop a row, add its index to "indexes_to_drop"
    ### if too many values are 'None'
    pct_match_none = sum(1 for val in match.values if val is None) / denom
    if pct_match_none > 0.15:
        n_15_none_match += 1

    if not home_team_att.empty and not away_team_att.empty:
        matches.iloc[index, home_column_indexes] = home_team_att.values[0]
        matches.iloc[index, away_column_indexes] = away_team_att.values[0]
        
        
        ### This is just an experiment to determine a threshold for how many values should be 'None'
        ### in team attribute data in order for us to drop a row. To drop a row, add its index to "indexes_to_drop"
        ### if too many values are 'None'
        pct_home_none = sum(1 for val in home_team_att.values[0] if val is None) / len(home_team_att.values)
        pct_away_none = sum(1 for val in away_team_att.values[0] if val is None) / len(away_team_att.values)
        if pct_home_none > 0.15 or pct_away_none > 0.3:
            n_15_none_team_att += 1
        
    else:
        indexes_to_drop.append(index)

### Part of our experiments
n_rows = index
print('total input rows:', n_rows)
print('num lacking any team attribute data:', len(indexes_to_drop))
print('num where >15% of team data is None:', n_15_none_match)
print('num where >15% of team attribute data is None:', n_15_none_team_att)

matches = matches.drop(indexes_to_drop, axis=0) ### Drops rows that lack too much data

print('Took {0:.2f} seconds.'.format(time.time() - st))

In [ ]:
#drop first 13 of matches:
matches = matches.drop(['date', 'season', 'home_team_goal', 'away_team_goal' ,'home_team_api_id',
 'away_team_api_id', 'goal', 'shoton' ,'shotoff', 'foulcommit', 'card', 'cross',
 'corner', 'possession'], axis=1)

In [ ]:
#Enumerate the columns if they have string values
newCol = {}
for col in matches.columns.values:
    if re.search('Class', col):
            #print(col, matches[col])
            #enum_dict = dict(enumerate(list(set(matches[col]))))
            enum_dict = { k: v for v, k in dict(enumerate(list(set(matches[col])))).items()}
            #print(col, enum_dict)
            #print(matches[col])
            newCol[col] = matches[col].map(enum_dict)
#print(newCol['__home_buildUpPlaySpeedClass'])
for colName in newCol.keys():
    matches[colName] = newCol[colName]

In [ ]:
# Get rid of cols missing betting odds
to_remove = []
columns = matches.columns.values[:30]
for index, match in matches.iterrows():
    n_missing = 0
    for col in columns: 
        if pd.isnull(match[col]):
            n_missing += 1
    if n_missing >= 15:
        to_remove.append(index)
        
matches = matches.drop(to_remove, axis=0)
matches = matches.drop(['__home_id', '__home_team_api_id', '__away_id', '__away_team_api_id'], axis=1)

In [ ]:
new_to_remove = []
for col in matches.columns.values:
    if re.search('date', col):
        new_to_remove.append(col)
matches = matches.drop(new_to_remove, axis=1)
print(matches.columns.values)

In [ ]:
filtered_og_matches = original_matches.copy()
filtered_og_matches = filtered_og_matches.loc[matches.index.values]

n_home_wins = len(filtered_og_matches[filtered_og_matches['home_team_goal'] > filtered_og_matches['away_team_goal']])
n_draws = len(filtered_og_matches[filtered_og_matches['home_team_goal'] == filtered_og_matches['away_team_goal']])
n_away_wins = len(filtered_og_matches[filtered_og_matches['home_team_goal'] < filtered_og_matches['away_team_goal']])
n_games = len(filtered_og_matches)

assert(n_games == n_home_wins + n_draws + n_away_wins)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
 
# Data to plot
labels = 'Home Wins', 'Draws', 'Away Wins'
sizes = [n_home_wins, n_draws, n_away_wins]
colors = ['red', 'lightcoral', 'grey']
explode = (0.1, 0, 0)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
 
plt.axis('equal')
plt.savefig('games_pie_chart.png')
plt.show()

In [ ]:
hist = pd.DataFrame.hist(filtered_og_matches['season_id'])

print(filtered_og_matches.columns.values)

hist